In [2]:
%%capture --no-stderr
%pip install --quiet -U langchain langchain_community tiktoken langchain-nomic "nomic[local]" langchain-ollama scikit-learn langgraph tavily-python bs4


[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
!ollama pull deepseek-r1:1.5b

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠙ pulling manifest 
pulling aabd4debf0c8...   0% ▕                ▏    0 B/1.1 GB                  pulling manifest 
pulling aabd4debf0c8...   0% ▕                ▏    0 B/1.1 GB                  pulling manifest 
pulling aabd4debf0c8...   0% ▕                ▏    0 B/1.1 GB                  pulling manifest 
pulling aabd4debf0c8...   0% ▕                ▏    0 B/1.1 GB                  pulling manifest 
pulling aabd4debf0c8...   0% ▕                ▏    0 B/1.1 GB                  pulling manifest 
pulling aabd4debf0c8...   0% ▕                ▏    0 B/1.1 GB                  

In [2]:
from langchain_ollama import ChatOllama

local_llm = "deepseek-r1:1.5b"
llm = ChatOllama(model=local_llm, temperature=0)
llm_json_mode = ChatOllama(model=local_llm, temperature=0, format="json")


In [4]:
import json
from langchain_community.vectorstores import SKLearnVectorStore
from langchain_nomic.embeddings import NomicEmbeddings

# Load the JSON file
with open("vectorstore.json", "r", encoding="utf-8") as f:
    vector_data = json.load(f)

# Reconstruct the vector store
vectorstore = SKLearnVectorStore.from_texts(
    texts=vector_data["documents"],
    embedding=NomicEmbeddings(model="nomic-embed-text-v1.5", inference_mode="local"),
    metadatas=vector_data["metadata"],
)

print("Vector store reloaded from JSON!")


Embedding texts: 100%|██████████| 16/16 [00:13<00:00,  1.20inputs/s]

Vector store reloaded from JSON!


In [ ]:
# from langchain.text_splitter import RecursiveCharacterTextSplitter
# from langchain_community.document_loaders import PyMuPDFLoader  # Load PDFs
# from langchain_community.vectorstores import SKLearnVectorStore
# from langchain_nomic.embeddings import NomicEmbeddings

# # Step 1: Load all PDFs in the current directory
# pdf_files = [f for f in os.listdir() if f.endswith(".pdf")]
# docs_list = []

# for pdf_path in pdf_files:
#     loader = PyMuPDFLoader(pdf_path)
#     docs_list.extend(loader.load())  # Append loaded documents

# # Step 2: Split documents
# text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
#     chunk_size=1024, chunk_overlap=200
# )
# doc_splits = text_splitter.split_documents(docs_list)

# # Step 3: Add to vectorDB
# vectorstore = SKLearnVectorStore.from_documents(
#     documents=doc_splits,
#     embedding=NomicEmbeddings(model="nomic-embed-text-v1.5", inference_mode="local"),
# )

# # Step 4: Create retriever (ensure k does not exceed available documents)
# retriever = vectorstore.as_retriever(k=min(2, len(doc_splits)))

# print(f"Loaded {len(doc_splits)} document chunks from {len(pdf_files)} PDFs.")



Embedding texts: 100%|██████████| 16/16 [00:11<00:00,  1.45inputs/s]

Loaded 16 document chunks from 2 PDFs.


In [5]:
retriever = vectorstore.as_retriever(k=3)

In [6]:
import os
import getpass
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Set API key from environment or prompt user
def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"Enter value for {var}: ")

# Ensure API key is set
_set_env("TAVILY_API_KEY")

# Set parallelism setting for tokenizers
os.environ["TOKENIZERS_PARALLELISM"] = "true"


In [7]:
### Router
import json
from langchain_core.messages import HumanMessage, SystemMessage

# Prompt
# router_instructions = """You are an expert at routing a user question to a vectorstore or web search.

# The vectorstore contains documents related to agents, prompt engineering, and adversarial attacks.

# Use the vectorstore for questions on these topics. For all else, and especially for current events, use web-search.

# Return JSON with single key, datasource, that is 'websearch' or 'vectorstore' depending on the question."""

router_instructions = """
You are an expert router for directing user queries.

The vectorstore contains documents related to:
- job role
- person qualification required
- skills

Use 'vectorstore' if the query is related to any of these topics.
For everything else (including current events), return 'websearch'.

### IMPORTANT ###
Your response MUST be a **valid JSON object** with exactly one key:
{
  "datasource": "vectorstore"
}
or
{
  "datasource": "websearch"
}

Do NOT include explanations, additional keys, or any other information.
"""


# Test router
test_web_search = llm_json_mode.invoke(
    [SystemMessage(content=router_instructions)]
    + [
        HumanMessage(
            content="What are the required skills for this job role?"
        )
    ]
)
# test_web_search_2 = llm_json_mode.invoke(
#     [SystemMessage(content=router_instructions)]
#     + [HumanMessage(content="What are the models released today for llama3.2?")]
# )
# test_vector_store = llm_json_mode.invoke(
#     [SystemMessage(content=router_instructions)]
#     + [HumanMessage(content="What are the types of agent memory?")]
# )
print(
    json.loads(test_web_search.content)
    # json.loads(test_web_search_2.content),
    # json.loads(test_vector_store.content),
)


{'datasource': 'vectorstore'}


In [8]:
### Retrieval Grader

# Doc grader instructions
doc_grader_instructions = """You are a grader assessing relevance of a retrieved document to a user question.

If the document contains keyword(s) or semantic meaning related to the question, grade it as relevant."""

# Grader prompt
doc_grader_prompt = """Here is the retrieved document: 

{document} 

Here is the user question: 

{question} 

Carefully and objectively assess whether the document contains at least some information that is relevant to the question.

### IMPORTANT ###
Your response MUST be a **valid JSON object** with exactly one key:
```json
{"binary_score": "Yes"}
"""

# Test
question = "What is skills required for this tescra job role?"
docs = retriever.invoke(question)

# if len(docs) == 0:
#     print("No relevant documents found.")
# elif len(docs) == 1:
#     print("Only one document retrieved.")
#     doc_txt = docs[0].page_content
# else:
#     doc_txt = docs[1].page_content



# if len(docs) < 2:
#     print("Warning: Not enough documents retrieved. Adjusting indexing.")
#     doc_txt = docs[0].page_content  # Fallback to first document
# else:
#     doc_txt = docs[1].page_content

doc_txt = docs[1].page_content
doc_grader_prompt_formatted = doc_grader_prompt.format(
    document=doc_txt, question=question
)
result = llm_json_mode.invoke(
    [SystemMessage(content=doc_grader_instructions)]
    + [HumanMessage(content=doc_grader_prompt_formatted)]
)
json.loads(result.content)

Embedding texts: 100%|██████████| 1/1 [00:00<00:00,  9.39inputs/s]


KeyError: '"binary_score"'

In [9]:
### Generate

# Prompt
rag_prompt = """You are an assistant for question-answering tasks. 

Here is the context to use to answer the question:

{context} 

Think carefully about the above context. 

Now, review the user question:

{question}

Provide an answer to this questions using only the above context. 

Use three sentences maximum and keep the answer concise.

Answer:"""


# Post-processing
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


# Test
question = "Is master's degree compulsory for st fox software role?"
docs = retriever.invoke(question)
docs_txt = format_docs(docs)
rag_prompt_formatted = rag_prompt.format(context=docs_txt, question=question)
generation = llm.invoke([HumanMessage(content=rag_prompt_formatted)])
print(generation.content)

Embedding texts: 100%|██████████| 1/1 [00:00<00:00,  8.92inputs/s]


<think>
Okay, so I need to figure out if a Master's degree is required for the St. Fox Software role based on the provided context. Let me read through the context again carefully.

The context mentions that the job requires a Bachelor's degree in Computer Science or a related field, with a Master's preferred. It also notes that prior experience in an agile development environment is beneficial. 

So, looking at that, it seems like the primary requirement is a Bachelor's degree, but they mention a Master's as a preferred option. Therefore, I think a Master's degree isn't strictly compulsory but is highly recommended.
</think>

Yes, a Master's degree is not required for the St. Fox Software role but is highly recommended due to the preference for it in the job description.


In [1]:
import json

# Extract vectors and document metadata
vector_data = {
    "documents": [doc.page_content for doc in doc_splits],
    "metadata": [doc.metadata for doc in doc_splits],
}

# Save as JSON
with open("vectorstore.json", "w", encoding="utf-8") as f:
    json.dump(vector_data, f, ensure_ascii=False, indent=4)

print("Vector store saved as JSON!")


NameError: name 'doc_splits' is not defined